In [2]:
import os
import pandas as pd

In [3]:
while "notebooks" in os.getcwd():
    os.chdir("../")

In [4]:
os.chdir("src")

In [5]:
from decision_tree.tree import parse_xgboost_dump, TreeEnsemble

In [6]:
trees = parse_xgboost_dump("../models/heloc-scaled-gbdt_dumped.txt")

In [7]:
some_trees = TreeEnsemble(trees.trees)
#some_trees = TreeEnsemble(trees.trees[0:10])

In [8]:
df = pd.read_csv("../data/heloc-scaled-test.csv")

In [9]:
df

,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,...,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance,RiskPerformance
0,-0.214900,1.699313,-0.056723,2.459516,-0.363267,1.201020,1.712719,-1.564641,-1.466360,-3.498869,...,-0.488720,-0.211494,-0.187532,-1.005586,0.090931,-0.702250,-0.955119,-0.062510,-1.379531,0.0
1,1.716386,-0.361454,-0.280331,0.499039,-0.185539,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,0.256357,-0.668264,-0.652592,-0.866162,-0.122189,-0.029287,-0.307994,-0.062510,0.391373,0.0
2,0.903213,0.638018,-0.578475,-0.027656,2.036062,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,1.852951,1.615589,1.672709,-1.075297,1.598802,-0.365768,-0.307994,-0.740481,-1.879017,1.0
3,-0.011607,-0.309935,-0.503939,-1.139568,-0.896451,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,-0.701599,0.245277,0.277528,0.946344,1.446950,0.307195,-0.307994,0.615461,0.981674,0.0
4,0.394979,0.586499,-0.503939,0.323474,1.680606,1.201020,0.658902,-0.119124,0.681985,0.147707,...,-0.062962,-0.668264,-0.652592,-0.238756,-0.096058,1.316640,-0.307994,-0.062510,-0.743822,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970,-0.926427,-1.731864,-0.429403,-1.871090,-0.896451,-0.476838,-0.394914,-0.884398,-1.495790,-1.067818,...,-0.169401,-0.668264,-0.652592,-0.761594,0.090931,-0.029287,-0.955119,-0.740481,-0.743822,0.0
1971,-0.621487,-0.608746,-0.056723,-0.964003,-1.429635,0.362091,0.658902,-0.799368,-0.377473,0.147707,...,-0.275841,0.702048,0.742588,0.493218,0.090931,-0.365768,0.056953,-0.062510,1.526568,0.0
1972,0.191686,-0.598442,-0.429403,-0.612873,-0.096675,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,1.852951,1.158818,1.207648,-0.308468,-1.438241,0.307195,0.339132,0.615461,-0.425967,0.0
1973,1.309799,0.205257,-0.578475,-0.964003,3.635614,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,-0.701599,0.245277,0.277528,-1.214721,-0.096058,-0.029287,0.339132,-0.740481,-2.060648,1.0


In [26]:
some_trees.eval(df.iloc[25])

-0.072109750269386

In [9]:
x = df.iloc[1]

In [10]:
x

ExternalRiskEstimate                  1.716386
MSinceOldestTradeOpen                -0.361454
MSinceMostRecentTradeOpen            -0.280331
AverageMInFile                        0.499039
NumSatisfactoryTrades                -0.185539
NumTrades60Ever2DerogPubRec          -0.476838
NumTrades90Ever2DerogPubRec          -0.394914
PercentTradesNeverDelq                0.646149
MSinceMostRecentDelq                  0.946849
MaxDelq2PublicRecLast12M              0.755470
MaxDelqEver                           0.876990
NumTotalTrades                       -0.280830
NumTradesOpeninLast12M               -0.465434
PercentInstallTrades                 -0.146751
MSinceMostRecentInqexcl7days          0.256357
NumInqLast6M                         -0.668264
NumInqLast6Mexcl7days                -0.652592
NetFractionRevolvingBurden           -0.866162
NetFractionInstallBurden             -0.122189
NumRevolvingTradesWBalance           -0.029287
NumInstallTradesWBalance             -0.307994
NumBank2NatlT

In [11]:
y = some_trees.eval(x)

In [12]:
y

-0.066468205765002

## Porównanko ewaluacji na pierwszych 20 datapointach

In [31]:
import xgboost as xgb
from pathlib import Path


In [32]:
model_path = Path("../models")
model_name = "heloc-scaled-gbdt"
gbdt_model = xgb.Booster()
gbdt_model.load_model(model_path / f"{model_name}_saved.json")


In [33]:
true_labels = df.iloc[:20,-1]


In [34]:
adam_trees_eval = [some_trees.eval(df.iloc[i]) for i in range(20)]


In [35]:
x_dmatrix = xgb.DMatrix(df.iloc[:20, :-1])
actual_xgb_model_eval = gbdt_model.predict(x_dmatrix)


In [36]:
pd.DataFrame(zip(true_labels, adam_trees_eval, actual_xgb_model_eval),
             columns=["RiskPerformance", "Eval by Adam Trees", "Actual eval by XGB model"])


,RiskPerformance,Eval by Adam Trees,Actual eval by XGB model
0,0.0,0.010588,0.033684
1,0.0,-0.066468,0.891780
2,1.0,0.007300,0.825676
3,0.0,0.141220,0.235195
4,1.0,-0.030481,0.982154
5,1.0,-0.063104,0.639026
6,0.0,-0.031091,0.304353
7,0.0,0.044860,0.358153
8,1.0,-0.065320,0.448349
9,0.0,-0.111742,0.128955


Widzimy że wyniki z dwóch ostatnich kolumn są różne, a powinny być dokładnie takie same.

In [13]:
from decision_tree.prediction_gap import NormalPredictionGap

In [14]:
pred_gap = NormalPredictionGap(1)

Powinno wyjść 0

In [15]:
pred_gap.prediction_gap_fixed(some_trees, x, y, set([]))

-4.53120275133688e-18

### PGI dla każdego pojedynczego featura i wybranego x: kwadrat ogólny, kwadrat szybki, moduł szybki

In [16]:
# %%time
# for column in df.columns:
#     print("%-40s %.15f %.15f %.15f" % (column,
#         pred_gap.prediction_gap_fixed(some_trees, x, y, set([column])),
#         pred_gap.prediction_gap_single_squared(some_trees, x, column, y),
#         pred_gap.prediction_gap_single_abs(some_trees, x, column, y)
#     ))

In [17]:
%%time
for column in df.columns:
    print("%-40s %.15f %.15f" % (column,
        # pred_gap.prediction_gap_fixed(some_trees, x, y, set([column])),
        pred_gap.prediction_gap_single_squared(some_trees, x, column, y),
        pred_gap.prediction_gap_single_abs(some_trees, x, column, y)
    ))

ExternalRiskEstimate                     0.000011308620171 0.000637991506204
MSinceOldestTradeOpen                    0.000053667392303 0.004166356900739
MSinceMostRecentTradeOpen                0.000000000000000 0.000000000000000
AverageMInFile                           0.000373340481467 0.010367724578639
NumSatisfactoryTrades                    0.000000064090500 0.000134956043549
NumTrades60Ever2DerogPubRec              0.000000024506582 0.000001399716838
NumTrades90Ever2DerogPubRec              0.000000000000000 0.000000000000000
PercentTradesNeverDelq                   0.000029548125910 0.002681696361051
MSinceMostRecentDelq                     0.000004131135881 0.000376590915077
MaxDelq2PublicRecLast12M                 0.000000000000000 0.000000000000000
MaxDelqEver                              0.000000000000000 0.000000000000000
NumTotalTrades                           0.000000000294372 0.000011167900623
NumTradesOpeninLast12M                   0.000000000011005 0.000000273336132

In [49]:
x = df.iloc[1]
y = some_trees.eval(x)


In [50]:
%%time
for column in df.columns:
    print("%-40s %.15f" % (column,
        pred_gap.prediction_gap_fixed(some_trees, x, y, set([column]))
        #pred_gap.prediction_gap_single_squared(some_trees, x, column, y),
        #pred_gap.prediction_gap_single_abs(some_trees, x, column, y)
    ))

ExternalRiskEstimate                     0.000011308620171
MSinceOldestTradeOpen                    0.000053667392303
MSinceMostRecentTradeOpen                -0.000000000000000
AverageMInFile                           0.000373340481466
NumSatisfactoryTrades                    0.000000064090500
NumTrades60Ever2DerogPubRec              0.000000024506582
NumTrades90Ever2DerogPubRec              -0.000000000000000
PercentTradesNeverDelq                   0.000029548125910
MSinceMostRecentDelq                     0.000004131135881
MaxDelq2PublicRecLast12M                 -0.000000000000000
MaxDelqEver                              -0.000000000000000
NumTotalTrades                           0.000000000294372
NumTradesOpeninLast12M                   0.000000000011005
PercentInstallTrades                     0.001620442533935
MSinceMostRecentInqexcl7days             0.000706823219876
NumInqLast6M                             0.000000978073744
NumInqLast6Mexcl7days                    0.000000004

### Poniżej przykład przeliczony na kartce (z innym rozkładem co prawda)

In [18]:
from decision_tree.tree import Split, Leaf

In [19]:
test_tree = TreeEnsemble([Split("dupa", 0, Split("dupa", -0.5, Leaf(-1), Leaf(5), Leaf(0)), Leaf(1), Leaf(0))])

In [20]:
x = {'dupa': 0.1}
y = test_tree.eval(x)

In [21]:
pred_gap.prediction_gap_fixed(test_tree, x, y, set(["dupa"]))

4.0717171905643

In [22]:
pred_gap.prediction_gap_single_squared(test_tree, x, "dupa", y)

4.071717190566653

### Global prediction-gap dla każdego feature'a przez 100 pierwszych punktow

Single squared liczy się 4 minuty dla 100 datapoints, single absolute również. Trochę głupio że liczymy prediction gap dla RiskPerformance, to jest wartość którą model ma przewidzieć, a nie jeden z feature'ów. Dobrze że wychodzi zero dla RiskPerformance. Próbowałem dać `x = df.iloc[i, :-1]` żeby wyłączyć RiskPerformance z iksa, ale wtedy wywala się.

In [27]:
%%time
results_sqr = []
for column in df.columns:
    glob = 0.0
    for i in range(100):
        x = df.iloc[i]
        glob += pred_gap.prediction_gap_single_squared(some_trees, x, column, some_trees.eval(x))
    glob /= df.shape[0]
    results_sqr.append((column, glob))
    print("%-40s %.15f" % (column, glob))


ExternalRiskEstimate                     0.000031488038478
MSinceOldestTradeOpen                    0.000002674123846
MSinceMostRecentTradeOpen                0.000000000057923
AverageMInFile                           0.000011713968308
NumSatisfactoryTrades                    0.000021494725862
NumTrades60Ever2DerogPubRec              0.000000063500947
NumTrades90Ever2DerogPubRec              0.000000000000184
PercentTradesNeverDelq                   0.000001175187395
MSinceMostRecentDelq                     0.000017414224730
MaxDelq2PublicRecLast12M                 0.000000000151656
MaxDelqEver                              0.000000000224679
NumTotalTrades                           0.000000045216312
NumTradesOpeninLast12M                   0.000002308314588
PercentInstallTrades                     0.000058383118675
MSinceMostRecentInqexcl7days             0.000031976181398
NumInqLast6M                             0.000015702023006
NumInqLast6Mexcl7days                    0.0000000001670

In [28]:
%%time
results_abs = []
for column in df.columns:
    glob = 0.0
    for i in range(100):
        x = df.iloc[i]
        glob += pred_gap.prediction_gap_single_abs(some_trees, x, column, some_trees.eval(x))
    glob /= df.shape[0]
    results_abs.append((column, glob))
    print("%-40s %.15f" % (column, glob))


ExternalRiskEstimate                     0.000634053356677
MSinceOldestTradeOpen                    0.000126184153001
MSinceMostRecentTradeOpen                0.000000310335768
AverageMInFile                           0.000398239597587
NumSatisfactoryTrades                    0.000294544205106
NumTrades60Ever2DerogPubRec              0.000003645103525
NumTrades90Ever2DerogPubRec              0.000000002280555
PercentTradesNeverDelq                   0.000069360206407
MSinceMostRecentDelq                     0.000339712598121
MaxDelq2PublicRecLast12M                 0.000000618326177
MaxDelqEver                              0.000000455002566
NumTotalTrades                           0.000006912332738
NumTradesOpeninLast12M                   0.000077912341684
PercentInstallTrades                     0.000842376527914
MSinceMostRecentInqexcl7days             0.000662065087155
NumInqLast6M                             0.000326698333263
NumInqLast6Mexcl7days                    0.0000008968314

In [33]:
df_results_sqr = pd.DataFrame(results_sqr, columns=["Feature", "PredGap"])
df_results_sqr.sort_values("PredGap", ascending=False)


,Feature,PredGap
13,PercentInstallTrades,5.838312e-05
14,MSinceMostRecentInqexcl7days,3.197618e-05
0,ExternalRiskEstimate,3.148804e-05
21,NumBank2NatlTradesWHighUtilization,2.962735e-05
18,NetFractionInstallBurden,2.257473e-05
19,NumRevolvingTradesWBalance,2.211338e-05
4,NumSatisfactoryTrades,2.149473e-05
8,MSinceMostRecentDelq,1.741422e-05
15,NumInqLast6M,1.570202e-05
3,AverageMInFile,1.171397e-05


In [34]:
df_results_abs = pd.DataFrame(results_abs, columns=["Feature", "PredGap"])
df_results_abs.sort_values("PredGap", ascending=False)


,Feature,PredGap
13,PercentInstallTrades,8.423765e-04
14,MSinceMostRecentInqexcl7days,6.620651e-04
0,ExternalRiskEstimate,6.340534e-04
21,NumBank2NatlTradesWHighUtilization,6.262501e-04
18,NetFractionInstallBurden,5.660030e-04
19,NumRevolvingTradesWBalance,4.392032e-04
3,AverageMInFile,3.982396e-04
8,MSinceMostRecentDelq,3.397126e-04
15,NumInqLast6M,3.266983e-04
4,NumSatisfactoryTrades,2.945442e-04


Widzimy drobne różnice pomiędzy kolejnością cech przy sortowaniu po pred gap kwadratowej a bezwzględnej. Jednakże sześć najważniejszych jest w dokładnie tej samej kolejności i żadna cecha nie przesuwa się o więcej niż 3 pozycje.

In [37]:
banzhaf_results = [5.3082e-01, 6.7819e-02, 1.0057e-03, 1.1615e-01, 7.9269e-02, 7.5800e-04,
        1.9180e-04, 4.1986e-02, 1.2749e-01, 3.8168e-03, 1.4541e-03, 3.7466e-02,
        1.3944e-03, 5.2826e-02, 2.1571e-01, 2.6424e-02, 7.8650e-03, 1.4590e-01,
        1.0155e+00, 6.1204e-03, 1.3284e-02, 1.1442e-01, 2.2595e-02]

In [62]:
df_results_banzhaf = pd.DataFrame(zip(df_results_abs.Feature, banzhaf_results),
                                  columns=["Feature", "banzhaf global importance"])

In [63]:
df_results_banzhaf


,Feature,banzhaf global importance
0,ExternalRiskEstimate,0.530820
1,MSinceOldestTradeOpen,0.067819
2,MSinceMostRecentTradeOpen,0.001006
3,AverageMInFile,0.116150
4,NumSatisfactoryTrades,0.079269
5,NumTrades60Ever2DerogPubRec,0.000758
6,NumTrades90Ever2DerogPubRec,0.000192
7,PercentTradesNeverDelq,0.041986
8,MSinceMostRecentDelq,0.127490
9,MaxDelq2PublicRecLast12M,0.003817


In [177]:
df_results_banzhaf.sort_values("banzhaf global importance", ascending=False)


,Feature,banzhaf global importance
18,NetFractionInstallBurden,1.015500
0,ExternalRiskEstimate,0.530820
14,MSinceMostRecentInqexcl7days,0.215710
17,NetFractionRevolvingBurden,0.145900
8,MSinceMostRecentDelq,0.127490
3,AverageMInFile,0.116150
21,NumBank2NatlTradesWHighUtilization,0.114420
4,NumSatisfactoryTrades,0.079269
1,MSinceOldestTradeOpen,0.067819
13,PercentInstallTrades,0.052826


Widzimy znaczne różnice pomiędzy tym, jak wygląda kolejność wyznaczona przez Banzhaf global importances a nasze single pred gap, jednakże nie są one zupełnie od czapy (oba zgadzają się że cechy 0, 3, 8, 14, 18, 21 należą do najważniejszych).

## Wyliczanie PGI przez samplowanie

In [92]:
from pathlib import Path
import xgboost as xgb
import numpy as np


In [72]:
model_path = Path("../models")
model_name = "heloc-scaled-gbdt"
gbdt_model = xgb.Booster()
gbdt_model.load_model(model_path / f"{model_name}_saved.json")


In [142]:
big_x = df.iloc[:100, :-1]
x_dmatrix = xgb.DMatrix(big_x)
f = gbdt_model.predict(x_dmatrix)


In [124]:
rng = np.random.default_rng(seed=2137)


In [157]:
banzhaf_top_four = {"NetFractionInstallBurden",
                    "ExternalRiskEstimate",
                    "MSinceMostRecentInqexcl7days",
                    "NetFractionRevolvingBurden"}

single_abs_predgap_top_four = {"PercentInstallTrades",
                               "MSinceMostRecentInqexcl7days",
                               "ExternalRiskEstimate",
                               "NumBank2NatlTradesWHighUtilization"}


In [194]:
banzhaf_bottom = set(df_results_banzhaf.sort_values("banzhaf global importance", ascending=False).iloc[-4:]["Feature"])
single_abs_predgap_bottom = set(df_results_abs.sort_values("PredGap", ascending=False).iloc[-5:-1]["Feature"])
display(banzhaf_bottom)
display(single_abs_predgap_bottom)
display(banzhaf_bottom & single_abs_predgap_bottom)

{'MSinceMostRecentTradeOpen',
 'NumTrades60Ever2DerogPubRec',
 'NumTrades90Ever2DerogPubRec',
 'NumTradesOpeninLast12M'}

{'MSinceMostRecentTradeOpen',
 'MaxDelq2PublicRecLast12M',
 'MaxDelqEver',
 'NumTrades90Ever2DerogPubRec'}

{'MSinceMostRecentTradeOpen', 'NumTrades90Ever2DerogPubRec'}

In [155]:
big_x

,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,...,PercentInstallTrades,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance
0,-0.214900,1.449528,-0.056723,2.652618,-0.363267,1.201020,1.712719,-1.564641,-1.466360,-3.498869,...,-1.310432,-0.488720,-0.211494,-0.187532,-1.005586,0.090931,-0.702250,-0.955119,-0.062510,-1.379531
1,1.716386,-0.507944,-0.280331,0.392342,-0.185539,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,-0.146751,0.256357,-0.668264,-0.652592,-0.866162,-0.122189,-0.029287,-0.307994,-0.062510,0.391373
2,0.903213,0.559022,-0.578475,-0.033595,2.036062,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,-0.811712,1.852951,1.615589,1.672709,-1.075297,1.598802,-0.365768,-0.307994,-0.740481,-1.879017
3,-0.011607,-0.347802,-0.503939,-0.935188,-0.896451,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,-0.423818,-0.701599,0.245277,0.277528,0.946344,1.446950,0.307195,-0.307994,0.615461,0.981674
4,0.394979,0.699262,-0.503939,0.234551,1.680606,1.201020,0.658902,-0.119124,0.681985,0.147707,...,-0.645472,-0.062962,-0.668264,-0.652592,-0.238756,-0.096058,1.316640,-0.307994,-0.062510,-0.743822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.598273,0.247426,0.166885,-1.934904,-1.518499,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,0.850689,0.894995,-0.668264,-0.652592,-1.214721,0.181515,-1.375213,-0.307994,-0.740481,-0.743822
96,-0.418194,0.401213,-0.354867,-0.801916,0.791966,-0.476838,-0.394914,0.391058,-1.289784,-1.067818,...,-0.645472,1.852951,-0.668264,-0.652592,-0.134188,1.042011,0.307195,0.986257,-0.062510,-0.970861
97,-1.434660,-2.110056,-0.578475,-1.958946,-1.518499,-0.476838,-0.394914,0.646149,0.946849,0.755470,...,0.850689,1.852951,0.245277,0.277528,2.305724,0.383984,-1.038732,0.339132,-0.062510,1.526568
98,-0.113254,-0.271524,-0.205795,-0.549136,-0.363267,-0.476838,-0.394914,-0.289185,-0.024320,0.147707,...,1.848130,0.043478,-0.668264,-0.652592,-0.413035,0.839541,-0.365768,0.339132,-0.062510,1.526568


In [146]:
def normal_perturbation(x, mu, stddev):
    perturbed_x = x.copy()
    perturbed_x[list(perturbed_columns)] += rng.normal(loc=mu, scale=stddev,
                                                       size=(len(x), len(perturbed_columns)))
    x_dmatrix = xgb.DMatrix(perturbed_x)
    return gbdt_model.predict(x_dmatrix)
    

In [202]:
def prediction_gap_by_random_sampling(x, y, perturbed_columns: set,
                                      seed: int = 42, num_iter: int = 100):
    rng = np.random.default_rng(seed=seed)
    result = np.zeros(len(x))
    for i in range(num_iter):
        perturbed_y = normal_perturbation(x, 0.0, 1.0)
        result += np.abs(perturbed_y - y)
    result /= num_iter
    return np.mean(result)


## PGI results, 4 most important features are perturbed

The results are very close to each other, they don't allow for the conclusion that predgap performs worse.

In [172]:
prediction_gap_by_random_sampling(big_x, f, banzhaf_top_four, num_iter = 1000)


0.020527125184666362

In [173]:
prediction_gap_by_random_sampling(big_x, f, single_abs_predgap_top_four, num_iter = 1000)


0.020478761257315053

In the calculations above the standard deviation was 0.1, below it was 1.

In [207]:
prediction_gap_by_random_sampling(big_x, f, banzhaf_top_four, num_iter = 200)

0.04139796506774146

In [208]:
prediction_gap_by_random_sampling(big_x, f, single_abs_predgap_top_four, num_iter = 200)

0.04208879088675603

## PGU results, 4 least important features are perturbed

In [200]:
prediction_gap_by_random_sampling(big_x, f, banzhaf_bottom, num_iter = 100)


0.02023525602128357

In [201]:
prediction_gap_by_random_sampling(big_x, f, single_abs_predgap_bottom, num_iter = 100)

0.02061016376819462

In [199]:
banzhaf_bottom

{'MSinceMostRecentTradeOpen',
 'NumTrades60Ever2DerogPubRec',
 'NumTrades90Ever2DerogPubRec',
 'NumTradesOpeninLast12M'}

In the calculations above the standard deviation was 0.1, below it was 1.

In [209]:
prediction_gap_by_random_sampling(big_x, f, banzhaf_bottom, num_iter = 200)

0.04187166352861096

In [210]:
prediction_gap_by_random_sampling(big_x, f, single_abs_predgap_bottom, num_iter = 200)

0.041873460233141674